In [1]:
import pandas as pd

In [2]:
pontuacao_f1 = [25,18,15,12,10,8,6,4,2,1,1,0] # O penultimo numero é a melhor volta e o último numero é a pontuação do pole.
pontuacao_f1_sprint = [8,7,6,5,4,3,2,1,0,0,0,0]
jogo = "f1_2024"

if "f1_2022" in jogo:
    path_resultados = "Y://corrida_inscritos_2023.xls"
    opcao = 1
    if opcao == 1:
        path_output_csv = 'Y://pontuacao_corrida_dos_inscritos_2023.csv'
    elif opcao == 2:
        path_output_csv = 'Y://pontuacao_corrida_dos_membros_2023.csv'
elif "f1_2023" in jogo:
    path_resultados = "Y://corrida_inscritos_2025.xls"
    opcao = 4
    if opcao == 1:
        path_output_csv = 'Y://pontuacao_corrida_dos_inscritos_2025.csv'
    elif (opcao == 2) or (opcao == 4):
        path_output_csv = 'Y://pontuacao_corrida_dos_membros_2025.csv'
elif "f1_2024" in jogo:
    path_resultados = "Y://corrida_inscritos_2025.xls"
    opcao = 2
    if opcao == 1:
        path_output_csv = 'Y://pontuacao_corrida_dos_inscritos_2025.csv'
    elif opcao == 2:
        path_output_csv = 'Y://pontuacao_corrida_dos_membros_2025.csv'
elif "assetto" in jogo:
    path_resultados = "Y://corrida_inscritos_2025.xls"
    opcao = 5
    path_output_csv = 'Y://pontuacao_corrida_dos_membros_2025.csv'       

In [3]:
if opcao == 1:
    df_resultados = pd.read_excel(path_resultados, sheet_name="crossplay")
elif opcao == 2: 
    df_resultados = pd.read_excel(path_resultados, sheet_name="membros")
elif opcao == 3:
    pontuacao_f1 = [25,18,15,12,10,8,6,4,2,1,1,2]
    df_resultados = pd.read_excel(path_resultados, sheet_name="f2")
elif opcao == 4:
    df_resultados = pd.read_excel(path_resultados, sheet_name="membros-23")
elif opcao == 5:
    df_resultados = pd.read_excel(path_resultados, sheet_name="membros-assetto")


In [4]:
df_resultados.drop(["Index"], axis=1, inplace=True)

In [5]:
df_resultados

,Belgica,Holanda,Italia,Baku,Singapura,Austin,Mexico,Interlagos,Las Vegas,Qatar,Abu Dhabi,Melbourne,Shanghai
0,O_peralta,Kova,Kova,sKuro,Kova,O_peralta,lucasracertv,CauaHenrique367,lucasracertv,BrPB Vitor,O_peralta,sKuro,lucasracetv
1,Zedatapioca,sKuro,sKuro,alissonzeira,CauaHenrique367,lucasracetv,sKuro,lucasracetv,Vinicius3107,lucasracertv,BrPB Vitor,CauaHenrique267,tomasglopes (I)
2,alissonzeira,O_peralta,F1 2024,TheFireKing21,sKuro,CauaHenrique367,F1 2024,sKuro,Juankzer,sKuro,lucasracertv,O_peralta99,sKuro
3,CauaHenrique367,alissonzeira,O_peralta,O_peralta,O_peralta,Zedatapioca,Rickofsizzler,Zedatapioca,sKuro,Vinicius3107,sKuro,alissonzeira,Vinicius3107
4,TheFireKing21,Cristiano Roque,Alienobserv,Rickofsizzler,Alienobserv,sKuro,O_peralta,Rickofsizzler,PedroAnd,Rickofsizzler,Rickofsizzler,DNF,alissonzeira
5,Thisgo_66,TheFireKing21,alissonzeira,DNF,Rickofsizzler,alissonzeira,alissonzeira,Vinicius3107,Rickofsizzler,O_peralta,Vinicius3107,DNF,CauaHenrique367
6,DNF,DNF,Rickofsizzler,DNF,alissonzeira,TheFireKing21,TheFireKing21,O_peralta,Allan Magalhães,F1 2024,alissonzeira,DNF,Isaquinho3pernas
7,DNF,DNF,TheFireKing21,DNF,TheFireKing21,Rickofsizzler,JulioCO,alissonzeira,O_peralta,alissonzeira,DNF,DNF,CoringaKil
8,DNF,DNF,JulioCO,DNF,MurmuringOne23,JulioCO,DNF,TheFireKing21,Bill25binho,JulioCO,DNF,DNF,ruanf12022
9,DNF,DNF,DNF,DNF,DNF,DNF,DNF,DNF,DNF,DNF,DNF,DNF,Eddie Peugeot


In [6]:
df_pontos = pd.DataFrame(columns=["Piloto", "Pontos"])

In [7]:
for etapa in list(df_resultados.columns):
    df_etapa = pd.DataFrame(columns=["Piloto", "Pontos"])
    if (etapa.endswith("-S1")) or (etapa.endswith("-S2")): #código para Sprint (Nome da etapa seria exemplo Bahrain-S)
        df_etapa["Pontos"] = pontuacao_f1_sprint
    else:
        df_etapa["Pontos"] = pontuacao_f1
    df_etapa["Piloto"] = [driver.upper() for driver in df_resultados[etapa].values]
    df_pontos = pd.concat([df_pontos, df_etapa], ignore_index=True)

In [8]:
df_resultados[etapa].values

array(['lucasracetv', 'tomasglopes (I)', 'sKuro', 'Vinicius3107',
       'alissonzeira', 'CauaHenrique367', 'Isaquinho3pernas',
       'CoringaKil', 'ruanf12022', 'Eddie Peugeot', 'tomasglopes (I)',
       'lucasracetv'], dtype=object)

#### Gerando pontuação

In [9]:
try:
    df_pontuacao = df_pontos.groupby("Piloto").sum().sort_values(by="Pontos", ascending=False).drop("DNF", axis=0)
    df_pontuacao["Lider"] = df_pontuacao["Pontos"].values[0]
    df_pontuacao["Diferenca"] = df_pontuacao["Pontos"] - df_pontuacao["Lider"]
    df_pontuacao.drop(["Lider"], axis=1, inplace=True)
    df_pontuacao
except KeyError: #nao achou dnf
    df_pontuacao = df_pontos.groupby("Piloto").sum().sort_values(by="Pontos", ascending=False)
    df_pontuacao["Lider"] = df_pontuacao["Pontos"].values[0]
    df_pontuacao["Diferenca"] = df_pontuacao["Pontos"] - df_pontuacao["Lider"]
    df_pontuacao.drop(["Lider"], axis=1, inplace=True)
    df_pontuacao

#### Gera pontuação extendida

In [10]:
for etapa in list(df_resultados.columns):
    pilotos = [driver.upper() for driver in df_resultados[etapa].values]
    df_pontuacao[etapa] = 0
    if (etapa.endswith("-S1")) or (etapa.endswith("-S2")):
        for eachDriver, points in zip(pilotos, pontuacao_f1_sprint):
            if eachDriver != 'DNF':
                if df_pontuacao.at[eachDriver, etapa] == 0:
                    df_pontuacao.at[eachDriver, etapa] = points
                else:
                    df_pontuacao.at[eachDriver, etapa] += points
    else:
        for eachDriver, points in zip(pilotos, pontuacao_f1):
            if eachDriver != 'DNF':
                if df_pontuacao.at[eachDriver, etapa] == 0:
                    df_pontuacao.at[eachDriver, etapa] = points
                else:
                    df_pontuacao.at[eachDriver, etapa] += points

In [11]:
# Reposicionando as colunas
novasColunas = list(df_resultados.columns)
novasColunas.append("Pontos")
novasColunas.append("Diferenca")
df_pontuacao = df_pontuacao[novasColunas]
df_pontuacao[:50]

,Belgica,Holanda,Italia,Baku,Singapura,Austin,Mexico,Interlagos,Las Vegas,Qatar,Abu Dhabi,Melbourne,Shanghai,Pontos,Diferenca
Piloto,,,,,,,,,,,,,,,
SKURO,0,18,18,26,16,10,18,15,12,15,12,26,15,201,0
O_PERALTA,26,15,12,12,12,25,10,6,4,8,25,0,0,155,-46
ALISSONZEIRA,15,12,8,18,6,8,8,4,0,4,6,12,10,111,-90
LUCASRACERTV,0,0,0,0,0,0,26,0,26,19,15,0,0,86,-115
CAUAHENRIQUE367,12,0,0,0,18,15,0,25,0,0,0,0,8,78,-123
RICKOFSIZZLER,0,0,6,10,8,4,12,10,8,10,10,0,0,78,-123
KOVA,0,26,26,0,25,0,0,0,0,0,0,0,0,77,-124
LUCASRACETV,0,0,0,0,0,19,0,19,0,0,0,0,25,63,-138
VINICIUS3107,0,0,0,0,0,0,0,8,18,12,8,0,12,58,-143


In [12]:
df_pontuacao[51:]

,Belgica,Holanda,Italia,Baku,Singapura,Austin,Mexico,Interlagos,Las Vegas,Qatar,Abu Dhabi,Melbourne,Shanghai,Pontos,Diferenca
Piloto,,,,,,,,,,,,,,,


In [13]:
df_pontuacao[["Pontos", "Diferenca"]][:50]

,Pontos,Diferenca
Piloto,,
SKURO,201,0
O_PERALTA,155,-46
ALISSONZEIRA,111,-90
LUCASRACERTV,86,-115
CAUAHENRIQUE367,78,-123
RICKOFSIZZLER,78,-123
KOVA,77,-124
LUCASRACETV,63,-138
VINICIUS3107,58,-143


In [14]:
df_pontuacao[["Pontos", "Diferenca"]][51:]

,Pontos,Diferenca
Piloto,,


In [15]:
if opcao == 1:
    df_pontuacao.to_csv(path_output_csv)
    print("Pontuação da Corrida dos Inscritos exportada com sucesso...")
elif opcao >= 2:
    df_pontuacao.to_csv(path_output_csv)
    print("Pontuação da Corrida dos Membros exportada com sucesso...")

Pontuação da Corrida dos Membros exportada com sucesso...


#### Exportando a tabela em HTML

In [16]:
# from  datetime import datetime

# # Criar um DataFrame de exemplo (substitua isso com seus próprios dados)
# numero_de_linhas = df_pontuacao.shape[0]
# numero_de_colunas = df_pontuacao.shape[1]

# html_table = df_pontuacao.to_html(index=True)

# if opcao == 1:
#     title = "Tabela Corrida dos Inscritos"
# elif opcao == 2:
#     title = "Tabela Corrida dos Membros"

# df_pontuacao = df_pontuacao.reset_index()
# df_pontuacao.index = range(1, numero_de_linhas + 1)
# html_table = df_pontuacao.to_html(index=False, escape=False)
# data_hoje = (datetime.now()).strftime("%d/%m/%Y %H:%M:%S")

# # Criar tabela HTML com CSS incorporado
# html_code = f'''
# <html>
# <head>
#     <meta charset="latin-1"> 
#     <link rel="stylesheet" type="text/css" href="styles.css">
# </head>
# <body>
#     <h4>Atualizado em: {data_hoje}</h4>
#     {html_table}
# </body>
# </html>
# '''

# # Escrever o HTML para um arquivo
# if opcao == 1:
#     nome_arquivo = 'index_corrida_inscritos.html'
# else:
#     nome_arquivo = 'index_corrida_membros.html'
    
# with open(f'{nome_arquivo}', 'w') as file:
#     file.write(html_code)

# print(f"Arquivo HTML exportado com sucesso: {nome_arquivo}.html")

### Calculo da pontuação mensal para o gift card  
Insira o intervalo das etapas equivalente ao mês que deseja calcular a pontuação.  
Exemplo: Se as etapas 3 e 4 forem as etapas realizadas no determinado mês, coloque entre colchetes '[2, 3]'

In [28]:
if (opcao >= 2) and ("f1_2024" in jogo):
    try:
        inicio_fim = {"f1-23": (10,12), "f1-24": (15,18), "assetto": (3,6)}
        mes = {1: "Janeiro", 2: "Fevereiro", 3: "Março", 4:"Abril",
               5: "Maio", 6:"Junho", 7: "Julho", 8: "Agosto",
               9: "Setembro", 10: "Outubro", 11: "Novembro", 12: "Dezembro"}
        if opcao == 2:
            if "f1_2024" in jogo:
                etapa_inicio = inicio_fim.get("f1-24")[0]
                etapa_fim = inicio_fim.get("f1-24")[1]
            else:    
                etapa_inicio = inicio_fim.get("f1-23")[0]
                etapa_fim = inicio_fim.get("f1-23")[1]
        elif opcao == 4:
            etapa_inicio = inicio_fim.get("f1-24")[0]
            etapa_fim = inicio_fim.get("f1-24")[1]
        elif opcao == 5:
            etapa_inicio = inicio_fim.get("assetto")[0]
            etapa_fim = inicio_fim.get("assetto")[1]
        # pontuacao_f1 = [25,18,15,12,10,8,6,4,2,1,1,2]
        # df_resultados = pd.read_excel(path_resultados, sheet_name="f2")
        mes_code = 4
        df_pontos = pd.DataFrame(columns=["Piloto", "Pontos"])

        for etapa in list(df_resultados.columns[etapa_inicio-1:etapa_fim]):
            df_etapa = pd.DataFrame(columns=["Piloto", "Pontos"])
            if etapa.endswith("-S1"): #código para Sprint (Nome da etapa seria exemplo Bahrain-S)
                df_etapa["Pontos"] = pontuacao_f1_sprint
            else:
                df_etapa["Pontos"] = pontuacao_f1
            df_etapa["Piloto"] = [driver.upper() for driver in df_resultados[etapa].values]
            df_pontos = pd.concat([df_pontos, df_etapa], ignore_index=True)

        try:
            df_pontuacao = df_pontos.groupby("Piloto").sum().sort_values(by="Pontos", ascending=False).drop("DNF", axis=0)
            df_pontuacao["Lider"] = df_pontuacao["Pontos"].values[0]
            df_pontuacao["Diferenca"] = df_pontuacao["Pontos"] - df_pontuacao["Lider"]
            df_pontuacao.drop(["Lider"], axis=1, inplace=True)
            df_pontuacao
        except KeyError: #nao achou dnf
            df_pontuacao = df_pontos.groupby("Piloto").sum().sort_values(by="Pontos", ascending=False)
            df_pontuacao["Lider"] = df_pontuacao["Pontos"].values[0]
            df_pontuacao["Diferenca"] = df_pontuacao["Pontos"] - df_pontuacao["Lider"]
            df_pontuacao.drop(["Lider"], axis=1, inplace=True)
            df_pontuacao

        for etapa in list(df_resultados.columns[etapa_inicio-1:etapa_fim]):
            pilotos = [driver.upper() for driver in df_resultados[etapa].values]
            df_pontuacao[etapa] = 0
            if (etapa.endswith("-S1")) or (etapa.endswith("-S2")):
                for eachDriver, points in zip(pilotos, pontuacao_f1_sprint):
                    if eachDriver != 'DNF':
                        if df_pontuacao.at[eachDriver, etapa] == 0:
                            df_pontuacao.at[eachDriver, etapa] = points
                        else:
                            df_pontuacao.at[eachDriver, etapa] += points
            else:
                for eachDriver, points in zip(pilotos, pontuacao_f1):
                    if eachDriver != 'DNF':
                        if df_pontuacao.at[eachDriver, etapa] == 0:
                            df_pontuacao.at[eachDriver, etapa] = points
                        else:
                            df_pontuacao.at[eachDriver, etapa] += points

        # Reposicionando as colunas
        novasColunas = list(df_resultados.columns[etapa_inicio-1:etapa_fim])
        novasColunas.append("Pontos")
        novasColunas.append("Diferenca")
        df_pontuacao = df_pontuacao[novasColunas]
        print(f"Pontuação dos membros do mês de {mes.get(mes_code)}")
        print(df_pontuacao[:50])
    except IndexError:
        print("Nenhuma etapa ainda no mês de {}".format(mes.get(mes_code)))
else:
    print("Pontuação mensal só é válida para a corrida dos membros!")

Pontuação dos membros do mês de Março
                  Melbourne  Shanghai Pontos Diferenca
Piloto                                                
SKURO                    26        15     41         0
LUCASRACETV               0        25     25       -16
ALISSONZEIRA             12        10     22       -19
TOMASGLOPES (I)           0        19     19       -22
CAUAHENRIQUE267          18         0     18       -23
O_PERALTA99              15         0     15       -26
VINICIUS3107              0        12     12       -29
CAUAHENRIQUE367           0         8      8       -33
ISAQUINHO3PERNAS          0         6      6       -35
CORINGAKIL                0         4      4       -37
RUANF12022                0         2      2       -39
EDDIE PEUGEOT             0         1      1       -40


In [25]:
jogo

'f1_2024'

#### Exportar HTML para corrida dos membros

In [19]:
# if opcao == 2:
#     from  datetime import datetime

#     # Criar um DataFrame de exemplo (substitua isso com seus próprios dados)
#     numero_de_linhas = df_pontuacao.shape[0]
#     numero_de_colunas = df_pontuacao.shape[1]

#     html_table = df_pontuacao.to_html(index=True)

#     if opcao == 1:
#         title = "Tabela Corrida dos Inscritos"
#     elif opcao == 2:
#         title = "Tabela Corrida dos Membros"

#     df_pontuacao = df_pontuacao.reset_index()
#     df_pontuacao.index = range(1, numero_de_linhas + 1)
#     html_table = df_pontuacao.to_html(index=False, escape=False)
#     data_hoje = (datetime.now()).strftime("%d/%m/%Y %H:%M:%S")

#     # Criar tabela HTML com CSS incorporado
#     html_code = f'''
#     <html>
#     <head>
#         <meta charset="latin-1">
#          <link rel="stylesheet" type="text/css" href="styles.css">
#     </head>
#     <body>
#         <h2>Pontuacao dos membros do mes de {mes.get(mes_code)}
#         <h4>Atualizado em: {data_hoje}</h4>
#         {html_table}
#     </body>
#     </html>
#     '''

#     # Escrever o HTML para um arquivo
#     if opcao == 2:
#         nome_arquivo = 'index_gift_mensal.html'

#     with open(f'{nome_arquivo}', 'w') as file:
#         file.write(html_code)

#     print("Arquivo HTML exportado com sucesso: index_gift_mensal.html")

#### Atualiza index com a corrida mais recente

In [20]:
corrida_inscritos_mais_recente_id = "1qmqoRFXDlI"

In [21]:
# html_code = f"""
# <!DOCTYPE html>
# <html>
#   <head>
# 	<meta charset="latin-1">
#     <meta name="viewport" content="width=device-width, initial-scale=1.0">
#     <title>Corrida dos Inscritos 2024</title>
# 	<link rel="stylesheet" type="text/css" href="styles.css">
#   </head>
#   <body>
  
#   <header>
#     <h1><a href='index.html'>Corrida dos Inscritos 2024</a></h1>
#   </header>
	
#     <!-- O <iframe> (e o player de vídeo) substituirão esta tag <div>. -->
#     <div id="player"></div>

#     <script>
#       var tag = document.createElement('script');
#       tag.src = "https://www.youtube.com/iframe_api";
#       var firstScriptTag = document.getElementsByTagName('script')[0];
#       firstScriptTag.parentNode.insertBefore(tag, firstScriptTag);

#       var player;
#       function onYouTubeIframeAPIReady() {{
#         player = new YT.Player('player', {{
#           height: '100%',
#           width: '100%',
#           videoId: '{corrida_inscritos_mais_recente_id}',
# 		  playerVars: {{ 'autoplay': 1, 'controls': 1 }},
#           events: {{
#             'onReady': onPlayerReady,   
#           }}
#         }});
# 		player.setPlaybackQuality('hd720')
#       }}

#       function onPlayerReady(event) {{
#         event.target.playVideo();
#       }}

#       var done = false;
#       function onPlayerStateChange(event) {{
#         if (event.data == YT.PlayerState.PLAYING && !done) {{
#           done = true;
#         }}
#       }}

#       function stopVideo() {{
#         player.stopVideo();
#       }}
	  
	  
#     </script>
#   </body>
# </html>
# """

# with open(f'ultima_corrida_inscritos.html', 'w') as file:
#         file.write(f'{html_code}')

#### Ligar o server em Python 

In [22]:
#!python -m http.server